# Домашнее задание 2

**ФИО:** 

**Дедлайн:** 19 декабря в 23:59

*Допустимо привысить дедлайн на два дня и сдавать до 21 декабря в 23:59. Это никак не будет наказываться, но сдать ДЗ после этого времени будет нельзя*

In [ ]:
!pip install transformers datasets==3.6.0 peft accelerate evaluate seqeval scikit-learn matplotlib seaborn conllu -q

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    DataCollatorForTokenClassification
)
from peft import get_peft_model, LoraConfig, TaskType, PrefixTuningConfig
import evaluate
from sklearn.metrics import classification_report, matthews_corrcoef
import time
import copy

In [ ]:
# Сиды для воспроизводимости результатов

torch.manual_seed(42)
np.random.seed(42)

## Задание 1 (4 балла)

Начнём с классической задачи классификации. Вам предстоит поработать с датасетом [CoLA](https://huggingface.co/datasets/nyu-mll/glue) (Corpus of Linguistic Acceptability), который используется для проверки способности модели отличать грамматически верные предложения от неверных. В качестве базовой модели возьмите `bert-base-uncased`.

Нужно провести два эксперимента по дообучению и сравнить их результаты:

1.  **Полное дообучение.** Обучите модель целиком на полном тренировочном наборе данных.
2.  **LoRA.** Обучите только адаптеры LoRA, также используя полный тренировочный набор.

**Требования к эксперименту:**

  * Обучайте модели как минимум 3 эпохи (можно больше, так как датасет небольшой и учится быстро).
  * Используйте валидационную выборку для оценки качества. Основные метрики здесь — [Matthews Correlation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html) и Accuracy. По желанию можно репортить и другие, например, F1-score.

**Анализ результатов:**

После обучения обеих моделей постройте графики или выведите сводную таблицу, где будут видны время обучения, количество обучаемых параметров и итоговое качество на валидации.

Ответьте на вопросы:

  * Насколько сильно отличается время обучения? Опционально можете сравнить потребления видеопамяти в момент обучения (можно посмотреть в Колабе)
  * Сильно ли уступает (или выигрывает) LoRA по качеству полной модели?
  * Как вы думаете, в каких ситуациях оправдано использование полного дообучения для таких задач, а в каких хватит адаптера?


## Задание 2 (4 балла)

В этом задании мы попытаемся выяснить, насколько хорошо лингвистические знания переносятся между разными языками. Для этого мы будем использовать задачу POS-теггинга (определение частей речи) и многоязычную модель `bert-base-multilingual-cased` (mBERT). С точки зрения машинного обучения мы решаем задачу классификации токенов (token classification), то есть назначаем класс для каждого слова в предложении, как в NER.

Суть эксперимента: вы обучаете модель определять части речи только на английском языке, а затем тестируете её качество на немецком языке (Zero-shot transfer). Модель никогда не видела размеченных немецких данных в процессе обучения.

Вам снова нужно реализовать два подхода:

1.  **Полное дообучение** mBERT на английском датасете.
2.  **LoRA** для mBERT на английском датасете.

**Важное примечание по данным:**

Мы будем использовать датасеты `universal_dependencies`. Чтобы не скачивать CONLL-U-файлы вручную и не приводить их к нужному формату, можно загрузить их в `datasets` с помощью уже подготовленных скриптов. Для этого версия библиотеки должна быть не выше 3.6.0, в более поздних версиях загрузка датасетов с помощью удалённых скриптов уже не работает. Также нужна библиотека `conllu`. Если вы запускали ячейку с установкой в начале тетрадки, там это уже учтено.


In [ ]:
ud_english = load_dataset(
    "universal_dependencies",
    "en_ewt",
    trust_remote_code=True)
ud_german = load_dataset(
    "universal_dependencies",
    "de_gsd",
    trust_remote_code=True)

ud_english

**Требования к эксперименту:**

  * Обучите обе версии модели на английских данных (минимум 3 эпохи).
  * Проверьте качество на английских валидационных данных.
  * Проверьте качество на немецком тестовом наборе (это и есть zero-shot).
  * Для оценки используйте F1-score и Accuracy.

**Анализ результатов:**

Сравните результаты английского и немецкого языков для обоих подходов. Ответьте на вопросы:

  * Насколько сильно падает качество при переходе на другой язык?
  * Есть ли разница в способности переносить знания между полной моделью и LoRA? Кто лучше справился с переносом?
  * Почему вообще этот перенос возможен? Какие лингвистические категории являются универсальными, а какие специфичны для языка, и как это влияет на результат?

## Задание 3\* (2 балла)

Здесь мы заглянем внутрь "черного ящика" и попробуем понять, что происходит с весами модели во время обучения в первых двух заданиях. Его удобно выполнять параллельно с первыми двумя.

**Что нужно сделать:**

1.  **Анализ весов.** Вам необходимо написать функцию для расчета Евклидова расстояния между весами исходной предобученной модели и весами дообученной модели. Нас интересует не просто одно число, а изменение весов на разных уровнях архитектуры. Эта функция должна принимать на вход исходную модель, дообученную модель и (опционально) имена слоёв, между которыми мы хотим посчитать разницу. Если имена слоёв отсутствуют, возвращаем среднее расстояние между всеми соответствующими слоями двух моделей, если есть конкретные слои — разницу только для указанных слоёв.

    Посчитайте расстояние между исходным и конечным состоянием для следующих компонентов:

      * Слой эмбеддингов.
      * Первый слой энкодера (нижний уровень).
      * Средний слой энкодера.
      * Последний слой энкодера (верхний уровень).
      * Слой классификатора (голова).

    Сделайте это сравнение для всех четырех моделей из предыдущих заданий (CoLA Full, CoLA LoRA, POS Full, POS LoRA). Для LoRA берите объединённую модель (база + адаптер), сравнивайте с базой. Можно построить гистограммы расстояний по слоям.

2.  **Кривые обучения (Learning Curves):**

    Визуализируйте графики потерь (loss) на тренировке и валидации по шагам или эпохам. Сравните динамику сходимости для полной модели и LoRA. Для этого вам может потребоваться увеличить количество эпох при обучении и сохранять метрики не каждую эпоху, а каждые *n* шагов.

**Ответьте на вопросы:**

  * Правда ли, что при дообучении основные изменения происходят только в верхних слоях трансформера? Подтверждают ли это ваши цифры?
  * Есть ли разница в распределении изменений весов между задачей классификации предложений (CoLA) и классификации токенов (POS)?
  * Почему слой классификатора (голова) обычно меняется сильнее всего?
  * Если смотреть на кривые обучения, есть ли смысл увеличивать количество эпох? Видите ли вы признаки переобучения?

## Расскажите нам, что думаете об этой домашке

Как вам задания? Насколько интересно? Насколько сложно? Что хотелось бы добавить, изменить или убрать? Только честно! :)